In [1]:
import torch
import numpy as np
import torch.nn as nn
from torch.autograd import Variable

In [98]:
#loading data
my_matrix = np.loadtxt(open("S4AfterNor.csv",encoding='UTF-8'),delimiter=",",skiprows=0)
X, y = my_matrix[:,:-1],my_matrix[:,-1]
Non_MI_data = np.load('Non_MI_data.npy')
Non_MI_label = np.load('Non_MI_label.npy')

In [117]:
y_observational=[]
X_train_MI = np.load('MI_train_Kmeans.npy')
X_val_MI = np.load('MI_val_Kmeans.npy')

# combine valdation dataset and trianing dataset to enlarge sample for training
X_observational = np.vstack((X_train_MI,X_val_MI))

for n in range(len(X_observational)):
    y_observational.append(1)

In [100]:
criterion = nn.BCELoss()

X_observational = Variable(torch.from_numpy(np.array(X_observational)).float(), requires_grad=True)
Y_observational = Variable(torch.tensor(y_observational)).float()
    
rand_Y = np.random.randint(0,1,(X_observational.size()[0],1))
Y_generated = Variable(torch.tensor(rand_Y)).float()
    

In [101]:
Y_observational=Y_observational.reshape(Y_observational.shape[0], 1)
Y_generated=Y_generated.reshape(Y_generated.shape[0], 1)

In [102]:
#GAN model design

torch.manual_seed(0)

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.xavier_uniform_(m.weight.data)
    elif classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data)
        nn.init.constant_(m.bias.data, 0.0)
    elif classname.find('BatchNorm1d') != -1:
        nn.init.normal_(m.weight.data)
        nn.init.constant_(m.bias.data, 0.0)
        
def init_weights(net_layer):
    try:
        net_layer.apply(weights_init_normal)
    except:
        raise NotImplementedError('weights initialization error')

def init_weights(net_layer):
    try:
        net_layer.apply(weights_init_normal)
    except:
        raise NotImplementedError('weights initialization error')


In [103]:
#G
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(382, 300),
                                     nn.BatchNorm1d(300),
                                     nn.ReLU(True),
                                     nn.Linear(300, 75),
                                     nn.BatchNorm1d(75),
                                     nn.ReLU(True))
        init_weights(self.encoder)
        
        self.decoder = nn.Sequential(nn.Linear(75, 150),
                                     nn.BatchNorm1d(150),
                                     nn.ReLU(True),
                                     nn.Linear(150, 382))
        init_weights(self.decoder)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = self.sigmoid(x)
        return x
        

In [104]:
#D
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.layer = nn.Sequential(nn.Linear(382, 200),
                                    nn.ReLU(True),
                                    nn.BatchNorm1d(200),
									nn.Linear(200, 200),
                                    nn.ReLU(True),
                                    nn.BatchNorm1d(200),
									nn.Linear(200, 200),
                                    nn.ReLU(True),
                                    nn.BatchNorm1d(200),
									nn.Linear(200, 100),
                                    nn.ReLU(True),
                                    nn.BatchNorm1d(100),
									nn.Linear(100, 100),
                                    nn.ReLU(True),
                                    nn.BatchNorm1d(100),
                                    nn.Linear(100, 1))
                                    
        init_weights(self.layer)
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.layer(x)
        x = self.sigmoid(x)
        return x


In [105]:
G = autoencoder()
D = FCNN()

g_optimizer = torch.optim.Adam(G.parameters(), lr = 0.005, weight_decay = 1e-4)
d_optimizer = torch.optim.Adam(D.parameters(), lr = 0.005, weight_decay = 1e-4)
current_epoch = 0
reconstructionLoss = nn.SmoothL1Loss()
criterion = nn.BCEWithLogitsLoss()
MSELoss = nn.MSELoss()

In [106]:
#training model
G_loss = []
G_ACC = []
D_loss = []
D_ACC = []
STEP = []
for step in range(3000):
    for step_g in range(2):
        
        generated_img = G(X_observational)
        generated_out = D(generated_img)
        g_loss_1 = reconstructionLoss(generated_img,X_observational)
        g_loss = criterion(generated_out,Y_observational)
        g_loss = g_loss_1+g_loss
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
       
        Results_G = 0 
        for i in D(generated_img).detach().numpy():
            if i > 0.5:
                Results_G += 1
        
        ACC_G = Results_G / (X_observational.size()[0])
    
    
    for step_d in range(1):
    
        observational_out = D(X_observational) 

        d_loss_observational = criterion(observational_out, Y_observational)
        generated_out = D(generated_img.detach()) 
        d_loss_generated = criterion(generated_out, Y_generated)

        d_loss = d_loss_observational + d_loss_generated 
        d_optimizer.zero_grad()  
        d_loss.backward() 
        d_optimizer.step()
        
        Results_D_observational = 0 
        Results = []
        for i in observational_out:
            if i > 0.5:
                Results.append(1)
            else:
                Results.append(0)
        
        correct = 0
        
        for i in range(len(Results)):
            if Results[i] == y[i]:
                correct += 1
        
        D_observational_accuracy = correct / float(len(y))
        
        Results_D_generated = 0
        for i in D(generated_img).detach().numpy():
            if i < 0.5:
                Results_D_generated += 1
        D_generated_accuracy = Results_D_generated/X_observational.size()[0]

        ACC_D =  (D_observational_accuracy+D_generated_accuracy)/2

    if step % 10 == 0:
        D_loss.append(d_loss.data)
        G_loss.append(g_loss.data)
        STEP.append(step)
    
    if step%500 == 0:
        generated_label_MI= []
        generated_data_MI = []

        for j in range(2):
            generated_data = Variable(torch.randn(X_observational.size()[0], X_observational.size()[1]))
            generated_data =  G(generated_data).detach().numpy()
            for i in generated_data:
                generated_data_MI.append(i)
                generated_label_MI.append(1)
        
        X_MI=[]
        y_MI=[]
        X_Non_MI = []
        y_Non_MI = []
        for n in range(len(y)):
            if y[n] == 1:
                X_MI.append(X[n])
                y_MI.append(1)
            else:
                X_Non_MI.append(X[n])
                y_Non_MI.append(0)
        np.save('MI_generated_data_'+'{}'.format(step)+'.npy', generated_data_MI)

In [107]:
generated_label_MI= []
generated_label_Non_MI= []
generated_data_MI_list = []
generated_data_Non_MI = []

In [108]:
#generate data
for i in range(5):
    generated_data_MI = []
    for j in range(8):
        generated_data = Variable(torch.randn(X_observational.size()[0], X_observational.size()[1]))
        generated_data =  G(generated_data).detach().numpy()
        for i in generated_data:
            generated_data_MI.append(i)
            generated_label_MI.append(1)
    generated_data_MI_list.append(generated_data_MI)

X_MI=[]
y_MI=[]
X_Non_MI = []
y_Non_MI = []
for n in range(len(y)):
    if y[n] == 1:
        X_MI.append(X[n])
        y_MI.append(1)
    else:
        X_Non_MI.append(X[n])
        y_Non_MI.append(0)

In [109]:
for j in range(8):
    generated_data = Variable(torch.randn(X_observational.size()[0], X_observational.size()[1]))
    generated_data =  G(generated_data).detach().numpy()
    for i in generated_data:
        generated_data_Non_MI.append(i)
        generated_label_Non_MI.append(1)

X_MI=[]
y_MI=[]
X_Non_MI = []
y_Non_MI = []
for n in range(len(y)):
    if y[n] == 1:
        X_MI.append(X[n])
        y_MI.append(1)
    else:
        X_Non_MI.append(X[n])
        y_Non_MI.append(0)

In [110]:
generated_data_MI = np.vstack(generated_data_MI_list)
generated_data_MI = generated_data_MI.astype('float32')

In [112]:
#save data and model
np.save('MI_generated_data.npy',generated_data_MI)
np.save('MI_generated_label.npy',generated_label_MI)
np.save('MI_generatedl_Non_MI.npy',generated_data_Non_MI)
np.save('MI_generated_label_Non_MI.npy',generated_label_Non_MI)

torch.save(G,'G.pkl')
torch.save(D,'D.pkl')

In [113]:
#loading model 
D = torch.load('D.pkl')
Non_MI_data = np.load('Non_MI_data.npy')
Non_MI_label = np.load('Non_MI_label.npy')
MI_data = np.load('MI_data.npy')
MI_label = np.load('MI_label.npy')

In [114]:
X_Non_MI = Variable(torch.from_numpy(np.array(Non_MI_data)).float(), requires_grad=True)
Y_Non_MI = Variable(torch.tensor(Non_MI_label)).float()
X_MI = Variable(torch.from_numpy(np.array(MI_data)).float(), requires_grad=True)
Y_MI = Variable(torch.tensor(MI_label)).float()

In [115]:
#calculate the performance of GAN
number = 0
for i in D(X_Non_MI).detach().numpy():
    if i < 0.5:
        number += 1
ACC_D = number / len(X_Non_MI)

number = 0
for i in D(X_MI).detach().numpy():
    if i > 0.5:
        number += 1
ACC_G = number / len(X_MI)

In [118]:
ACC_D_G = np.array([ACC_D*ACC_G])
np.save('ACC_D_G.npy',ACC_D_G)
